<a href="https://www.kaggle.com/code/joymachadolab/brandmr-model-usecase-demo?scriptVersionId=248515517" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
! pip install huggingface
! pip install huggingface-hub
! pip install dataset
! pip install transformer
! pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.1 MB/s eta 0:00:0000:010:01
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.
langchain 0.3.12 requires async-timeout<5.0.0,>=4.0.0; python_version < "3.11", but you have async-timeout 5.0.1 which is incompatible.
ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 21.8 MB/s eta 0:00:00:00:0100:01


In [2]:
from huggingface_hub import login 
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
# Import kaggle secrets
from kaggle_secrets import UserSecretsClient
import pandas as pd


In [3]:
user_secrets = UserSecretsClient()

hf = user_secrets.get_secret("huggingface")

In [4]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

tokenizer = AutoTokenizer.from_pretrained("jmachado-lab/DeepSeek-R1-BrandMR-casetype-classification")
model = AutoModelForCausalLM.from_pretrained("jmachado-lab/DeepSeek-R1-BrandMR-casetype-classification", trust_remote_code=True, low_cpu_mem_usage=True).to("cuda")


tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

Exception ignored in: <function _xla_gc_callback at 0x7a42b33a24d0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

In [5]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Identity()
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
          

In [18]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are an intake employee for a Dutch legal firm expert with advanced knowledge in assigning incoming requests from users to the right kind of case types set by the firm.
The case types set by the legal firm are listed as: {}
Answer the following question sent in by a user in the Dutch language and classify the user question to the right kind of case type set by the firm along with your chain of thought. 
If there is more than 1 case type that's relevant to a situation, pick the one that is 
most relevant by reasoning why and stick to the case types set by the firm.
Assign atleast 1 case type with reasoning

### Question:
{}

### Response:
<think>
{}
</think>
{}
"""

In [23]:
df = pd.read_excel("../input/casetype-dataset/casetype_dataset.xlsx")
column_values = set(df["Zaaktype benaming"])
case_type_list = ", ".join(map(str, column_values))

In [21]:
question = "Ik was een werknemer in een winkel en ik werd beschuldigd van diefstal en ontslagen van mijn baan. Ik heb niets gestolen, daar ben ik zeker van. Kunt u mij helpen?"

inputs = tokenizer([train_prompt_style.format(case_type_list, question, "", "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
valid_response = response[0].split("### Response:")[1]
print(valid_response)


<think>

</think>

De beschuldiging van diefstal en ontslag op staande voet is een arbeidsrechtelijke kwestie. Dit past onder de AS ARBEIDSRECHT Overig, omdat het gaat om een arbeidsrelatie waarbij de werkgever een ontslag pleegt. Hier zijn geen civielrechtelijke procedures aan de hand, maar de focus is op de arbeidsrechtelijke behandeling van de situatie.<｜end▁of▁sentence｜>


In [24]:
question = "Ik was werknemer in een winkel en werd beschuldigd van diefstal en op staande voet ontslagen. Ik heb niets gestolen, daar ben ik zeker van. Kunt u mij helpen?"

inputs = tokenizer([train_prompt_style.format(case_type_list, question, "", "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
valid_response = response[0].split("### Response:")[1]
print(valid_response)


<think>

</think>

Ik was werknemer in een winkel en werd beschuldigd van diefstal en op staande voet ontslagen. Ik heb niets gestolen, daar ben ik zeker van. Kunt u mij helpen?

AS ARBEIDSRECHT Ontslag op staande voet<｜end▁of▁sentence｜>


In [25]:
question = "Ik heb via een website een schilder ingehuurd voor een dag om mijn huis te schilderen en heb de helft vooruitbetaald, maar hij is niet gekomen of heeft niet op mijn bericht gereageerd. Kunt u mij helpen mijn geld terug te krijgen van het bedrijf waar hij via is ingehuurd?"

inputs = tokenizer([train_prompt_style.format(case_type_list, question, "", "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
valid_response = response[0].split("### Response:")[1]
print(valid_response)


<think>

</think>

Ik heb via een website een schilder ingehuurd voor een dag om mijn huis te schilderen en heb de helft vooruitbetaald, maar hij is niet gekomen of heeft niet op mijn bericht gereageerd. Kunt u mij helpen mijn geld terug te krijgen van het bedrijf waar hij via is ingehuurd?

De situatie betreft een contractueel geschil over een niet voltooide dienst, waarbij geld is betaald voor een dienst die niet is ontvangen. Dit past onder het case type CA gekochte dienst. 

**Classificatie:** CA gekochte dienst<｜end▁of▁sentence｜>


In [11]:
question = "Ik heb een schilder ingehuurd om een ​​kamer in mijn huis te schilderen. We hebben e-mails heen en weer gestuurd en ik heb hem het voorschot rechtstreeks op zijn bankrekening gestort. Op de dag van de overeenkomst kwam hij niet opdagen en beantwoordde mijn telefoontjes niet. Is er iets wat we kunnen doen?"

inputs = tokenizer([train_prompt_style.format(case_type_list, question, "", "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
valid_response = response[0].split("### Response:")[1]
print(valid_response)


<think>

</think>

Ik heb een schilder ingehuurd om een kamer te schilderen. Na de overeenkomst, waarop ik hem het voorschot rechtstreeks op zijn bankrekening overmaakte, kwam hij niet opdagen en beantwoordde mijn telefoontjes niet. Dit is vervelend, maar kan ik hier rechtelijke actie tegen taken? 

Ik denk dat hier een conflict bestaat tussen de schilder en mij. Omdat de schilder de overeenkomst niet nakomt, kan ik mogelijk een geschil hebben met hem. Dit zou kunnen worden geklasseerd onder de categorie **CA gekochte dienst**, omdat de dienst niet is geleverd zoals overeengekomen is.<｜end▁of▁sentence｜>


In [12]:
question = "Ik heb een auto gekocht bij een erkende dealer, binnen een paar dagen na mijn aankoop, merkte ik problemen met mijn auto, waaronder problemen met de motor. Wat kan ik doen om de auto terug te brengen naar de dealer?"

inputs = tokenizer([train_prompt_style.format(case_type_list, question, "", "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
valid_response = response[0].split("### Response:")[1]
print(valid_response)


<think>

</think>

CA Auto<｜end▁of▁sentence｜>


In [13]:
question = "Het plafond van mijn huurhuis lekt en mijn verhuurder zou het moeten repareren, maar hij doet niets om het te repareren zoals in ons huurcontract staat. Welke stappen kan ik ondernemen om hem het probleem te laten oplossen?"

inputs = tokenizer([train_prompt_style.format(case_type_list, question, "", "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
valid_response = response[0].split("### Response:")[1]
print(valid_response)


<think>

</think>

Ik kan een ingebouwde opstelling maken aan de verhuurder om het plafond te laten repareren, zoals het contract meemaakt. Dit is de eerste stap. Als de verhuurder niet reageert, kan ik een aanrijding naar de rechtbank maken om het probleem te laten oplossen. Dit past onder de categorie WO Bouwrecht Aanneming van werk, omdat het gaat om een verplicht werk dat de verhuurder niet volgens het contract uitvoert.<｜end▁of▁sentence｜>


In [14]:
question = "Ik heb een conflict met de verkoper van een tweedehands boot. Wat kan ik doen?"

inputs = tokenizer([train_prompt_style.format(case_type_list, question, "", "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
valid_response = response[0].split("### Response:")[1]
print(valid_response)


<think>

</think>

Ik heb een conflict met de verkoper van een tweedehands boot. Wat kan ik doen?

### Case Type:
CA Consumentengeschil Verkopen en kopen<｜end▁of▁sentence｜>


In [15]:
question = "Ik ben ontslagen bij mijn baan in de VS. Kan ik juridische stappen ondernemen tegen mijn bedrijf?"

inputs = tokenizer([train_prompt_style.format(case_type_list, question, "", "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
valid_response = response[0].split("### Response:")[1]
print(valid_response)


<think>

</think>

De ontslag in de VS heeft gevolgen voor mijn arbeidsrechtelijke situatie in de NL. Ik wil juridische stappen ondernemen tegen mijn bedrijf, maar ik ben onzeker over de geschilpunten en procedure. 

Aangezien ik een Amerikaans ontslag heb, moet ik overwegen waar dit ontslag is geregeld en of dit rechtstreek op de NL arbeidsrechten van toepassing is. Het is mogelijk dat mijn rechten in de NL worden beïntefsd door het buitenlandse ontslag. 

Ik denk dat het AS ARBEIDSRECHT ziekte / arbeidsongeschiktheid zou kunnen zijn, omdat ik een ontslag op staande voet heb, maar ik ben niet ziek. Dit type case kan helpen om mijn rechten te versterken in de context van het ontslag in de VS.<｜end▁of▁sentence｜>


In [16]:
question = "Ik heb reisplannen gemaakt met een bedrijf in NL naar Mexico. Op het laatste moment hebben ze mijn plannen geannuleerd zonder goede reden. Ik heb geld uitgegeven, behalve het betalen van het reisbedrijf om de reis te plannen. Kan ik mijn geld terugkrijgen?"

inputs = tokenizer([train_prompt_style.format(case_type_list, question, "", "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
valid_response = response[0].split("### Response:")[1]
print(valid_response)


<think>

</think>

Ik heb reisplannen gemaakt met een bedrijf in NL naar Mexico. Op het laatste moment hebben ze mijn plannen geannuleerd zonder goede reden. Ik heb geld uitgegeven, behalve het betalen van het reisbedrijf om de reis te plannen. Kan ik mijn geld terugkrijgen?

CA gekochte dienst<｜end▁of▁sentence｜>


In [27]:
question = "Ik ben vandaag ontslagen"

inputs = tokenizer([train_prompt_style.format(case_type_list, question, "", "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
valid_response = response[0].split("### Response:")[1]
print(valid_response)


<think>

</think>

Ik ben vandaag ontslagen  
AS ARBEIDSRECHT Ontslag UWV<｜end▁of▁sentence｜>


In [29]:
question = "I have to fire a worker today, what steps can I take"

inputs = tokenizer([train_prompt_style.format(case_type_list, question, "", "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
valid_response = response[0].split("### Response:")[1]
print(valid_response)


<think>

</think>

De ontslag van een werknemer is een serieus onderdeel en moet volgens de arbeidsrechtelijke regels worden behandeld. Als werkgever moet je een duidelijke procedure volgen, waaronder het toesturen van een ontslagbrief met de daad verkeerende omstandigheden, het overleggen van de arbeidsvoorwaarden en het bestek van een ontslag op staande voet of via UWV. Raadpleeg een arbeidsrechtexpert voor een precieze behandeling van jouw specifieke situatie.<｜end▁of▁sentence｜>
